In [34]:
# HIDDEN
from ipywidgets import widgets, interact, Dropdown,SelectMultiple, Checkbox,VBox,FloatSlider,Button
from IPython.display import display
#https://stackoverflow.com/questions/18398929/passing-multiple-variables-from-dropdowns-using-django-python
#https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916
#https://medium.com/@jdchipox/how-to-interact-with-jupyter-33a98686f24e
#https://stackoverflow.com/questions/26352555/how-does-one-get-widget-values-with-a-button-in-ipython
#https://elc.github.io/posts/embed-interactive-notebooks/

In [35]:
# HIDDEN 
import numpy as np
import pandas as pd
import re
import os
from IPython import get_ipython
ipython = get_ipython()

In [36]:
df1 = pd.read_pickle('CH_aug.p')

In [37]:
# HIDDEN
style = {'description_width': 'initial'}
mnths = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}
cntry = np.load('trandict.npy',allow_pickle='TRUE').item()
cntry = {v:k for k,v in cntry.items()}

# Choose your country and the month for which you'd like a diet

In [38]:
# HIDDEN
var1 = widgets.Dropdown(options = cntry.keys(),value = 'Switzerland')
display(var1)
var2 = widgets.Dropdown(options = mnths.keys(),value='aug')
display(var2)

Dropdown(index=26, options=('Aruba', 'Angola', 'Anguilla', 'Albania', 'Andorra', 'Argentina', 'American Samoa'…

Dropdown(index=7, options=('jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec')…

In [39]:
# HIDDEN
runcode2 = widgets.Button(description='Build the database')
output2 = widgets.Output()
def test(b):
    global dftest2
    with output2:
        print ('I want a diet for:',var1.value, '\nfor the month of:',var2.value)
        ipython.magic('run Build_Optimization_Database.ipynb')
        dftest2 = calc_impacts(cntry[var1.value],var2.value)
        return dftest2

In [40]:
display(runcode2,output2)
runcode2.on_click(test)

Button(description='Build the database', style=ButtonStyle())

Output()

In [52]:
len(dftest2)

549

In [47]:
# HIDDEN
gender={'female':1,'male':0}
preferences = {'vegetarian':0,'vegan':1,'none':2}
indicator = {'climate change':'GHG','biodiversity loss':'BIO','both':'any'}
activity = {'sedentary':0,'moderate':1,'vigorous':2}
supp = {'yes':'yes_sup','no':'no_sup'}
weighting = {'climate change':0,'biodiversity loss':0,'both':0.5}
def weighting(impact):
    if impact == 'GHG' or 'BIO': w = 0
    elif impact == 'both':w = 0.5
    else: w = 0
    return w

# Please answer the following questions to personalize the diet to your needs

In [48]:
#HIDDEN
personweight = widgets.IntText(description='Weight (kg):',value=60)
display(personweight)

imp = widgets.Dropdown(options=indicator.values(),description='Impact')
display(imp)
gen = widgets.Dropdown(options=gender.values(),description='Gender:')
display(gen)
height = widgets.IntText(description='Height (cm):',value=165)
display(height)
age = widgets.IntText(description='Age (yrs):',value=40)
display(age)
act = widgets.Dropdown(options=activity.keys(),description='Activity Level:',value='moderate')
display(act)
sup = widgets.Dropdown(options=supp.values(),description='Supplements?',style=style)
display(sup)
days = widgets.BoundedIntText(description='Number of Days:',style=style,value=7)
display(days)
diettype = widgets.Dropdown(options=preferences.keys(),description='Dietary Preferences:',style=style,value='none')
display(diettype)


IntText(value=60, description='Weight (kg):')

Dropdown(description='Impact', options=('GHG', 'BIO', 'any'), value='GHG')

Dropdown(description='Gender:', options=(1, 0), value=1)

IntText(value=165, description='Height (cm):')

IntText(value=40, description='Age (yrs):')

Dropdown(description='Activity Level:', index=1, options=('sedentary', 'moderate', 'vigorous'), value='moderat…

Dropdown(description='Supplements?', options=('yes_sup', 'no_sup'), style=DescriptionStyle(description_width='…

BoundedIntText(value=7, description='Number of Days:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Dietary Preferences:', index=2, options=('vegetarian', 'vegan', 'none'), style=Descripti…

In [84]:
# HIDDEN

def joinlists_new(widgetname):
    global selectedfoods
    rs1 = [widgetname.description for widgetname in widgetname.children[1].children if widgetname.value]
    rs2 = [widgetname.value for widgetname in widgetname.children[2].children if widgetname.value]
    food = list(zip(rs1,rs2))
    
    try:
        food
        for j in food:
            selectedfoods[j[0].split(',')[0]] = {}#'names'
            selectedfoods[j[0].split(',')[0]]['names']=j[0]
            selectedfoods[j[0].split(',')[0]]['mass']=j[1]
    except NameError:
        i= None   
    return #selectedfoods

# Are there certain foods you must have?

In [126]:
# HIDDEN
listofcheckboxes = {'no preferences':[0],'fruits':['FAT'],'vegetables':['DGR','DGC'],
     'legumes':['DBR','DBC','DFR','DFC'],'grains':['AC','AC_wg','DAR','DAC',
     'AI','AI_wg','AA','AA_wg','AF','AF_wg','AD','AD_wg','AC','AC_wg'],'nuts_seeds':
     ['GAT'],'milk':['BAE','BAH','BAK','BLS','BLM','BLH','BLF','BN'],
     'vegetarianproducts':['BAV','BNV','VEG'],
     'fish':['JAC', 'JAR', 'JCC', 'JCR', 'JKC', 'JKR', 'JMC', 'JMR', 'JRC'],
    'meat':['MBGR','MBGC','MACR','MACC','MAER','MECC','MECR','MEER','MEEC','MAIC','MAIR','MAE',\
    'MAIR','MAIC','MCOR','MCOC','MCAR','MCAC','MAGR','MAGC','MAHR','MAHC',
    'MI','MAAC','MAAR'],'eggs_oil':['CA','OA', 'OC', 'OE','BTM','BJC']}

In [125]:
# HIDDEN
selectedfoods = {}
#Generate a dummy list
Allfileslist = list(zip(df_foods['Food Name'],df_foods['Group']))
#Search box + generate ALL the checkboxes, this seems to be the bottleneck
search_widget = widgets.Dropdown(options = listofcheckboxes.keys(),layout={'width': 'initial'})
experiments = {}
experiments_mass = {}
options_widget = widgets.VBox(layout={'overflow': 'auto'})
mass_widget = widgets.VBox(layout={'overflow': 'auto'})
button_wid = widgets.Button(description='Save Choices')
but_out = widgets.Output()
def save(b):
    with but_out:
        joinlists_new(multi_select)
button_wid.on_click(save)    
    
def whentextischanged (change):
    """Dynamically update the widget experiments"""
    search_input = change['new']
    if search_input == 'no preferences':
        # Reset search field, default to last 9 experiments
        new_options = [widgets.Checkbox(description='Proceed with optimization', value=True)]
        new_options_mass = [widgets.Checkbox(description='Proceed with optimization', value=True)]
    else:
        # Filter by search
        close_matches = [x[0] for x in Allfileslist if x[1] in(listofcheckboxes[search_widget.value])][0:5]
        for name in close_matches:
            if name not in experiments:
                experiments[name] = widgets.Checkbox(description=name, value=False,style={'description_width': 'initial'})
                experiments_mass[name] = widgets.IntSlider(description='Mass (grams)', min=0,max=500,step=50,value=0)
        ### magic code that generates only the match boxes ###
        new_options = [experiments[eachfilename] for eachfilename in close_matches]
        new_options_mass = [experiments_mass[eachfilename] for eachfilename in close_matches]
    options_widget.children = new_options
    mass_widget.children = new_options_mass

In [127]:
# HIDDEN
#Generate the vbox, search
multi_select = widgets.HBox([search_widget, options_widget,mass_widget,button_wid])
search_widget.observe(whentextischanged, names='value')
#serve it up
multi_select

In [49]:
# HIDDEN
runcodeA = widgets.Button(description='Foods I have to have')
outputA = widgets.Output()
def includefoods(b):
    global df_foods
    with outputA:
        ipython.magic('run Foods_to_Include.ipynb')
        df_foods = editdf(dftest2,imp.value,diettype.value,sup.value)
        return df_foods
display(runcodeA,outputA)
runcodeA.on_click(includefoods)

Button(description='Foods I have to have', style=ButtonStyle())

Output()

In [123]:
runcode5 = widgets.Button(description='Run Tool')
output5 = widgets.Output()
def run_tool(b):
    global df_opt
    with output5:
        
        
        
        ipython.magic('run Optimization_Tool_Online.ipynb')
        print ('This can take some time.')
        df_opt = runopt(df_foods,imp.value,diettype.value,sup.value,gen.value,personweight.value,\
                          height.value,age.value,act.value,days.value,2,selectedfoods)
        return 
display(runcode5,output5)
runcode5.on_click(run_tool)

Button(description='Run Tool', style=ButtonStyle())

Output()

In [124]:
# HIDDEN
df_opt[(df_opt['Grams']!=0)&(df_opt['Grams'].notnull())]

,Food Name,Grams,optimization_country_GHG_1_trade,optimization_value_GHG_1_trade,bio_GHGopt_value1,optimization_value_BIO_1_trade,optimization_country_BIO_1_trade,GHG_bioopt_value1,bio_GHGopt_value1
0,"almonds,_flaked_and_ground_GAT",500.000000,US,0.003466,2.132229e-17,1.124133e-17,AU,0.006993,2.132229e-17
2,"apples,_eating,_raw,_flesh_and_skin,_weighed_w...",500.000000,IT,0.000205,1.005305e-18,1.274994e-19,CH,0.000349,1.005305e-18
13,"avocado,_flesh_only,_average,_weighed_with_ski...",500.000000,IL,0.001170,2.091114e-18,2.026653e-18,ZA,0.001512,2.091114e-18
25,"sunflower_seeds,_toasted_GAT",500.000000,CH,0.000639,2.871373e-18,2.603193e-18,DE,0.000925,2.871373e-18
26,"bananas,_raw,_flesh_only,_weighed_with_skin_FAT",500.000000,CO,0.000637,7.015127e-18,4.155707e-18,PA,0.000732,7.015127e-18
47,"beans,_broad,_dried,_raw,_canned_DBR",420.000000,CH,0.000332,3.850140e-17,5.920336e-19,GB,0.000580,3.850140e-17
86,"bread,_wholemeal,_average_AF_wg",420.000000,CH,0.000477,1.321229e-18,1.439377e-19,FI,0.001030,1.321229e-18
98,"cereal,_malted_flake,_fortified_AI",420.000000,CH,0.000750,1.771647e-18,4.775524e-19,GB,0.001488,1.771647e-18
113,"pasta,_wholewheat,_spaghetti,_dried,_raw_AD_wg",357.097470,CH,0.000587,1.516410e-18,8.235970e-19,DE,0.000814,1.516410e-18
148,sauerkraut_DGR,539.360230,CH,0.000364,5.100528e-19,2.174748e-19,NL,0.000481,5.100528e-19


In [129]:
import matplotlib

In [ ]:
def choices(foodgroup):
    selectedfoods = {}
    try:
        variable = foodgroup
    except NameError:
        pass    
    else:    
        for j in variable:
            selectedfoods[j[0].split(',')[0]] = {}#'names'
            selectedfoods[j[0].split(',')[0]]['names']=j[0]
            selectedfoods[j[0].split(',')[0]]['mass']=j[1]

    return selectedfoods